In [4]:
import pandas as pd
import json


In [5]:
DB_DATA_PATH = "data/HungThinhDB10K.csv"

db_data = pd.read_csv(DB_DATA_PATH)


In [6]:
with open('data/intent_alias_data.json', 'r', encoding="utf-8") as fp:
    alias_file = json.load(fp)
    fp.close()


In [7]:
def preprocess(old_data, old_column_name, new_column_name):
    temp_data = pd.DataFrame(old_data)
    temp_data = temp_data.rename(columns={old_column_name: new_column_name})
    temp_data = temp_data.dropna()
    temp_data = temp_data.drop_duplicates()

    for x in temp_data.index:
        if "http" in temp_data.loc[x, new_column_name]:
            # print(x, temp_data.loc[x, new_column_name])
            temp_data = temp_data.drop(x)

        # if "/" in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
            # temp_data = temp_data.drop(x)

        # if "." in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
            # temp_data = temp_data.drop(x)

        # if "<" in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
        #     temp_data = temp_data.drop(x)

        # if ">" in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
        #     temp_data = temp_data.drop(x)

        # if "01" in temp_data.loc[x, new_column_name]:
        #     print(x, temp_data.loc[x, new_column_name])
        #     temp_data = temp_data.drop(x)

    return temp_data


In [17]:
def add_alias_to_csv(file_csv_path, new_alias_column_name, alias_list):
    temp_data = pd.read_csv(file_csv_path)
    new_column = pd.DataFrame({new_alias_column_name: alias_list})
    temp_data = temp_data.merge(
        new_column, how="left", left_index=True, right_index=True)
    temp_data.to_csv(file_csv_path, index=False)


In [10]:
def create_alias_column(file_csv_path, new_alias_column_name, alias_file, alias_data):
    alias_list = alias_file[alias_data]
    add_alias_to_csv(file_csv_path, new_alias_column_name, alias_list)


In [11]:
def create_csv_file(db_data, column_name, new_column_name, file_name):
    temp_data_old = db_data[[column_name]]
    new_data = preprocess(temp_data_old, column_name, new_column_name)
    new_data.to_csv(file_name)


In [139]:
# implement house
create_csv_file(db_data, "real_estate_sub_type",
                "house_type", "data/house_data.csv")
create_alias_column("data/house_data.csv", "house_alias",
                    alias_file, "house_type")


In [13]:
# implement city
create_csv_file(db_data, "city", "city_name", "data/city_data.csv")



In [18]:
create_alias_column("data/city_data.csv", "city_alias", alias_file, "city")

In [143]:
# implement district
create_csv_file(db_data, "district", "district_name", "data/district_data.csv")

In [ ]:
create_alias_column("data/district_data.csv",
                    "district_alias", alias_file, "district")


In [145]:
# implement ward
create_csv_file(db_data, "ward", "ward_name", "data/ward_data.csv")
create_alias_column("data/ward_data.csv", "ward_alias", alias_file, "ward")


In [146]:
# implement street
create_csv_file(db_data, "street", "street_name", "data/street_data.csv")
create_alias_column("data/street_data.csv",
                    "street_alias", alias_file, "street")


In [147]:
# implement house_number
create_csv_file(db_data, "house_number", "house_number",
                "data/house_number_data.csv")
create_alias_column("data/house_number_data.csv",
                    "house_number_alias", alias_file, "house_number")
